# Subnotebook to 01_seti_end_to_end.ipynb for raw file execution

#### Raw mode initialization 1

In [ ]:
if raw_mode:

    delete_raw_file = False    
    raw_file_base_name = raw_dir + raw_file_stem + '.0000.raw'
    raw_file_delete_spec = raw_dir + raw_file_stem + '.00*.raw'

    local_raw_file_found = False
    print(raw_file_base_name)
    if os.path.isfile(raw_file_base_name): 
        print('Raw file found in raw directory')
        local_raw_file_found = True
    else:
        raw_file_backup_name = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.0000.raw'
        raw_file_backup_spec = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.00*.raw'
        if os.path.isfile(raw_file_backup_name): 
            if enable_raw_backup_copy:
                print('Raw file found in backup directory, copying')
                t1 = time.time()
                if (0):     # single raw file .0000.raw
                    print(raw_file_backup_name)
                    os.system('cp '+raw_file_backup_name+' '+raw_dir)
                    os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
                else:       # multi raw file .00xx.raw
                    os.system('ls '+raw_file_backup_spec)
                    os.system('cp '+raw_file_backup_spec+' '+raw_dir)
                    os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
                t_copy = time.time() - t1
                print(f'Copy complete, {t_copy=:.2f} sec\n')
                os.system('ls -lsah '+raw_dir+'*.raw')
                delete_raw_file = delete_raw_file_enable
            else:
                print('Raw file found in backup directory, no copy, read directly')
                print(raw_file_backup_name)
                raw_dir = raw_backup_base_dir + raw_backup_subdir
                raw_file_base_name = raw_dir + raw_file_stem + '.0000.raw'
                delete_raw_file = False
        else:
            print('Raw file not found in backup directory, skip')
            print(raw_file_backup_name)
            # stop, or return to calling notebook/script
            assert(False)

    mjd_int = file_name_mjd(raw_file_stem)
    if (mjd_int < 58000):
        if (fine_fft_size%1009 > 0):
            print(f'Warning: {fine_fft_size=} doesn''t evenly divide block size for early raw files\n')

#### Raw mode initialization 2

In [ ]:
if raw_mode:

    print(raw_file_stem)
    raw_parts_list = raw_file_stem.split('-')
    print(raw_parts_list)

    if (isChirp):
        telescope = raw_parts_list[0]
        signal_level = float(raw_parts_list[2][0:-1])
        n_chirp = int(raw_parts_list[1][5:])
        f_start1_MHz = float(raw_parts_list[3])
        f_stop1_MHz = float(raw_parts_list[4])
        if (raw_parts_list[5][0]=='m'):
            sig_min_drift = -float(raw_parts_list[5][1:])  # remove minus sign with 'm' for negative drift limits in raw file name
        else:
            sig_min_drift = float(raw_parts_list[5])
        if (raw_parts_list[6][0]=='m'):
            sig_max_drift = -float(raw_parts_list[6][1:])  # remove minus sign with 'm' for negative drift limits in raw file name
        else:
            sig_max_drift = float(raw_parts_list[6])
        print(f'{n_chirp = } {f_start1_MHz = } {f_stop1_MHz = } {sig_min_drift = } {sig_max_drift = }')
            

    elif(isVoyager):
        telescope = ''
        signal_level = np.nan
        n_chirp = 2
        if (0):
            delta_f1_MHz = 1.5
            f_start1_MHz = 8420.432 - delta_f1_MHz
            f_stop1_MHz = 8420.432 + delta_f1_MHz
        else:
            delta_f1_MHz = np.nan
            f_start1_MHz = np.nan
            f_stop1_MHz = np.nan

        sig_min_drift = search_min_drift
        sig_max_drift = search_max_drift
    elif(isguppi):
        telescope = ''
        signal_level = np.nan
        n_chirp = 2
        delta_f1_MHz = np.nan
        f_start1_MHz = np.nan
        f_stop1_MHz = np.nan
        
        sig_min_drift = search_min_drift
        sig_max_drift = search_max_drift
        

    f_start_truth = np.linspace(f_start1_MHz*1e6,f_stop1_MHz*1e6,n_chirp)
    drift_rate_truth = np.linspace(sig_min_drift,sig_max_drift,n_chirp)
    f_start_incr = f_start_truth[1]-f_start_truth[0]
    df_dt_incr = drift_rate_truth[1]-drift_rate_truth[0]

    if verbose:
        print(f'{signal_level = }, {n_chirp = }')
        print(f'{f_start1_MHz = }, {f_stop1_MHz = }, { f_start_incr = }, {df_dt_incr = }')
        print(f'{f_start_truth[0] = } {f_start_truth[-1] = }')
        print(f'{drift_rate_truth[0] = } {drift_rate_truth[-1] = }')

    


#### Raw mode initialization 3

In [ ]:
if raw_mode:
    import src.get_raw_info as raw
    p,header = raw.get_run_params(raw_file_stem,raw_dir,fine_fft_size,n_sti)
    print(p)
    # exec(open("src/get_raw_info.py").read())
    # copy p entries into local variables
    # %run -i "src/raw_info_script.py"
    n_lti = p['n_lti']
    n_sti = p['n_sti']
    n_avg = p['n_avg']

    if (n_lti<=2):
        print(f'Note: {n_lti=}, skipping execution\n')
        # stop, or return to calling notebook/script
        assert(False)

    if not telescope:
        telescope = p['telescop']
        
    f_min_MHz = p['f_min_MHz']
    f_max_MHz = p['f_max_MHz']
    t_obs = p['t_obs']
    

#### Raw mode initialization 4

In [ ]:
if raw_mode:
    n_coarse_channels = p['n_coarse_channels']
    snr_input_db = 10*np.log10(signal_level**2 /2)  # 1 polarization
    snr_coarse_db = snr_input_db + 10*np.log10(n_coarse_channels)
    snr_fine_db = snr_coarse_db + 10*np.log10(fine_fft_size)
    snr_best_db = snr_fine_db + 5*np.log10(2.*n_avg)
    mean_window_loss = 1.11 # mean loss due to rectangular window within FFT bin

    print('signal_level={0:6.4f}, Input SNR={1:4.2f} Coarse SNR={2:4.2f} Fine SNR={3:4.2f} Det SNR={4:4.2f} dB'.format(\
            signal_level,snr_input_db,snr_coarse_db,snr_fine_db,snr_best_db))

    ref_snr_db = snr_best_db - mean_window_loss  # subtract mean window loss
    print('signal_level={0:6.4f}, Best SNR={1:4.2f}, Reference SNR={2:4.2f} dB'.format(signal_level,snr_best_db,ref_snr_db))
    reference_str = f'Reference={ref_snr_db:.2f} dB'

    # open directory for figures if necessary
    if not os.path.isdir(fig_dir[0:-1]):
        os.system('mkdir '+fig_dir[0:-1])
    # empty old figures

    config_str = 'Nfft=%.0fK, Nsti=%d, Nlti=%.0f, %.1f sec  %s'%(fine_fft_size/1024,n_sti,n_lti,t_obs,test_case)
    config_str2 = '%.0fK-Nsti-%02d'%(fine_fft_size/1024,n_sti)
    if verbose:print(config_str,'\n',config_str2)
    fig_name_base = raw_file_stem + '-' + config_str2
    if verbose: print(f'{fig_name_base = }')

    if math.isnan(f_start1_MHz):
        f_start1_MHz = p['ctr_freq_MHz'] - 2*p['fs_coarse']*1e-6
        f_stop1_MHz = p['ctr_freq_MHz'] + 2*p['fs_coarse']*1e-6
        fig_f_limits_MHz = [f_start1_MHz,f_stop1_MHz]
    else:
        fig_df = (np.amax(f_start_truth)*1e-6 - np.amin(f_start_truth)*1e-6)*.40
        fig_f_limits_MHz = [np.amin(f_start_truth)*1e-6 - fig_df,np.amax(f_start_truth)*1e-6 + fig_df]

    if verbose:
        with np.printoptions(precision=4, suppress=True):
            print(f'f_start_truth = {f_start_truth[0:3]*1e-6} ... {f_start_truth[-3:]*1e-6} MHz')
            print(f'drift_rate_truth = {drift_rate_truth[0:3]} ... {drift_rate_truth[-3:]} Hz/sec')
            print(f'figure freq limits MHz: {np.array(fig_f_limits_MHz)} ')

#### Raw Mode: Generate spectrogram file using rawspec

In [ ]:
if raw_mode:
    if (fb_ext == '.h5'):
        raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -j ' + raw_dir + raw_file_stem + ' -d ' + sg_dir
        # raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -s 3 -n 1 -j ' + raw_dir + raw_file_stem
    elif (fb_ext == '.fil'):
        raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -j ' + raw_dir + raw_file_stem + ' -d ' + sg_dir
    if verbose: print(raw_call_string + '\n')
    t1 = time.time()
    os.system(raw_call_string)
    t_rawspec0 = time.time() - t1
    t_rawspec = t_rawspec0/n_coarse_channels
    print(f'\nRawspec {t_rawspec0:.2f} sec {t_rawspec:.2f} sec/coarse channel')

    file_stats = os.stat(raw_file_base_name)
    raw_size_MB = file_stats.st_size/1024/1024
    raw_size_MB_per_chnl = raw_size_MB/n_coarse_channels
    if verbose: print(f'raw File size = {raw_size_MB:6.0f} MB x {p["n_raw_files"]}, {raw_size_MB_per_chnl:6.0f} MB per Coarse Channel')


#### Raw Mode: Augment file name of filterbank h5/fil file to include parameters (fft, n_sti)

In [ ]:
if raw_mode:
    import glob
    import os
    h5_name_list = glob.glob(sg_dir + raw_file_stem + '.*' + fb_ext)
    print(sg_dir + raw_file_stem + '.*' + fb_ext)
    print('h5_name_list=')
    print(f'{h5_name_list}')

    for i_name, h5_name in enumerate(h5_name_list):
        if (0):
            #use the same name to avoid excessive disk storage space (same file overwritten many times)
            new_h5_name = h5_name
        else:
            h5_parts = h5_name.split('.rawspec')
            print(f'{h5_parts = }')
            new_h5_name = h5_parts[0] + '-%.0fK-Nsti-%02d'%(fine_fft_size/1024,n_sti) + '.rawspec' + h5_parts[1]
            os.system('mv ' + h5_name + ' ' + new_h5_name)
        if (i_name==0):
            full_base_h5_name = new_h5_name
        if verbose: print(f'{new_h5_name = }')
        

    if verbose:
        print(f'{full_base_h5_name = }')

    print(new_h5_name)
    file_stats = os.stat(new_h5_name)
    h5_size_MB = file_stats.st_size/1024/1024
    h5_size_MB_per_chnl = h5_size_MB/n_coarse_channels
    if verbose: print(f'H5 File size = {h5_size_MB:6.0f} MB {h5_size_MB_per_chnl:6.0f} MB per Coarse Channel')